<a href="https://colab.research.google.com/github/ANANTHMANOJ/Ai_Projs/blob/master/logistic_regression_updated_dummy_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [307]:
import numpy as np 
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from torch import autograd

In [308]:
data = pd.read_csv("Iris.csv")
data.Species.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [309]:
data.shape

(150, 6)

In [310]:
data.Species = data.Species.replace(to_replace=['Iris-setosa', 'Iris-versicolor','Iris-virginica'], value=[0, 1, 2])

In [311]:
X = data.drop(labels=['Id', 'Species'], axis=1).values
y = data.Species.values

In [312]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [313]:
seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)

In [314]:
train_index = np.random.choice(len(X), round(len(X) * 0.8), replace=False)

In [315]:
test_index = np.array(list(set(range(len(X))) - set(train_index)))
train_X = X[train_index]
train_y = y[train_index]
test_X = X[test_index]
test_y = y[test_index]

In [316]:
def min_max_normalized(data):
    col_max = np.max(data, axis=0)
    col_min = np.min(data, axis=0)
    return np.divide(data - col_min, col_max - col_min)

In [317]:
train_X = min_max_normalized(train_X)
test_X = min_max_normalized(test_X)

In [318]:
train_X = torch.tensor(train_X)
train_y = torch.tensor(train_y)
test_X = torch.tensor(test_X)
test_y = torch.tensor(test_y)

In [319]:
W = torch.rand(size=(4, 3),requires_grad=True)
b = torch.rand(3, requires_grad=True)
params = [W, b]
params

[tensor([[0.6383, 0.2983, 0.1608],
         [0.0198, 0.2852, 0.6497],
         [0.3888, 0.2868, 0.5106],
         [0.5584, 0.8322, 0.9047]], requires_grad=True),
 tensor([0.7580, 0.6612, 0.5667], requires_grad=True)]

In [320]:
def softmax(y_linear):
    exp = torch.exp(y_linear-torch.max(y_linear))
    norms = torch.sum(exp, axis=1).reshape((-1,1))
    return exp / norms

In [321]:
sample_y_linear = torch.rand(size=(4,3))
sample_yhat = softmax(sample_y_linear)
print(sample_yhat)

tensor([[0.3898, 0.2706, 0.3396],
        [0.3401, 0.2692, 0.3907],
        [0.4189, 0.3201, 0.2611],
        [0.2708, 0.3523, 0.3769]])


In [322]:
torch.sum(sample_yhat,axis=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000])

In [323]:
def net(X):
    y_linear = torch.mm(X, W) + b
    yhat = softmax(y_linear)
    return yhat

In [324]:
def cross_entropy(yhat, y):
    return - torch.sum(y * torch.log(yhat+1e-6))

In [325]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad


In [326]:
def one_hot_encode(vector):
    n_classes = len(vector.unique())
    one_hot = torch.zeros((vector.shape[0], n_classes)).type(torch.LongTensor)
    return one_hot.scatter(1, vector.type(torch.LongTensor).unsqueeze(1), 1)

In [327]:
train_loader = torch.utils.data.DataLoader(dataset=list(zip(train_X,train_y)), batch_size=50, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=list(zip(test_X,test_y)), batch_size=50, shuffle=False)

In [328]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data
        label = label
        label_one_hot = one_hot_encode(label)
        output = net(data.float())
        predictions = torch.argmax(output, axis=1)
        numerator += torch.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator)

In [329]:
evaluate_accuracy(test_loader, net)


tensor(0.4000)

In [330]:
learning_rate = .005
epochs = 150
for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_loader):
        label_one_hot =one_hot_encode(label)
        #with autograd.:
        output = net(data.float())
        loss = cross_entropy(output, label_one_hot)
        loss.backward()
        with torch.no_grad():
          SGD(params, learning_rate)

        cumulative_loss += torch.sum(loss)


    test_accuracy = evaluate_accuracy(test_loader, net)
    train_accuracy = evaluate_accuracy(train_loader, net)
    print("Epoch %s. Loss: %.6s, Train_acc %.6s, Test_acc %.6s" % (e, float(cumulative_loss)/50, float(train_accuracy), float(test_accuracy)))

Epoch 0. Loss: 2.6148, Train_acc 0.4833, Test_acc 0.6000
Epoch 1. Loss: 2.3843, Train_acc 0.8916, Test_acc 0.8999
Epoch 2. Loss: 2.0312, Train_acc 0.9333, Test_acc 0.8666
Epoch 3. Loss: 1.6857, Train_acc 0.7916, Test_acc 0.8000
Epoch 4. Loss: 1.3482, Train_acc 0.9250, Test_acc 0.8999
Epoch 5. Loss: 1.1254, Train_acc 0.9166, Test_acc 0.8999
Epoch 6. Loss: 0.9612, Train_acc 0.9416, Test_acc 0.9333
Epoch 7. Loss: 0.8457, Train_acc 0.9250, Test_acc 0.9333
Epoch 8. Loss: 0.7477, Train_acc 0.9333, Test_acc 0.9333
Epoch 9. Loss: 0.6686, Train_acc 0.9333, Test_acc 0.9333
Epoch 10. Loss: 0.6006, Train_acc 0.9499, Test_acc 0.8999
Epoch 11. Loss: 0.5492, Train_acc 0.9499, Test_acc 0.8999
Epoch 12. Loss: 0.5089, Train_acc 0.9499, Test_acc 0.9333
Epoch 13. Loss: 0.4655, Train_acc 0.9499, Test_acc 0.9666
Epoch 14. Loss: 0.4332, Train_acc 0.9499, Test_acc 0.9666
Epoch 15. Loss: 0.4067, Train_acc 0.9499, Test_acc 0.9666
Epoch 16. Loss: 0.3858, Train_acc 0.9499, Test_acc 0.9666
Epoch 17. Loss: 0.3634, 

In [331]:
params

[tensor([[ -49.0244,   24.4024,   25.7194],
         [  84.2513,  -11.7738,  -71.5225],
         [-108.7943,    0.8393,  109.1417],
         [-105.7495,  -10.8288,  118.8734]], requires_grad=True),
 tensor([ 51.2962,  44.9304, -94.2407], requires_grad=True)]

In [332]:
y_one_hot = one_hot_encode(train_y)

In [333]:
def model_predict(net,data):
    output = net(data.float())
    return torch.argmax(output, axis=1)

sample_data = torch.utils.data.DataLoader(dataset=list(zip(train_X,train_y)), batch_size=5, shuffle=True)
for i, (data, label) in enumerate(sample_data):
    print(data.shape)
    pred=model_predict(net,data)
    print('model predictions are:', pred)
    print('Real values are : ',label)
    break

torch.Size([5, 4])
model predictions are: tensor([0, 1, 1, 0, 1])
Real values are :  tensor([0, 1, 1, 0, 1])
